# Easy GPT-Q + LoRA in JAX ([github](https://github.com/davisyoshida/easy-lora-and-gptq))

[Davis Yoshida](https://github.com/davisyoshida/)

This notebook shows how to combine  two JAX tools/transforms I wrote: [Lorax](https://github.com/davisyoshida/lorax) and [JAX-GPTQ](https://github.com/davisyoshida/jax-gptq). I've been using the combination to run LLaMA finetunes on a single GPU.

They're both applicable to basically any JAX function, which conveniently includes many HuggingFace models!

The procedure is as follows:

1. Quantize the weights of the model we want to use
2. Use Lorax to transform the original model function `F(params, inputs)` to one that takes a tuple of the original params and the low rank LoRA params: `F_lora(param_tuple, inputs)`
3. Wrap `F_lora` in `use_quantized` transform so that it knows how to handle arguments which are int8 matrices with two parameters per byte.
4. Train the model, updating only the low rank params and leaving the larger 4-bit model weights frozen.

I'd love feedback on one or both of these tools so please let me know on their Githubs if you have any suggestions. JAX-GPTQ in particular is still in a really early state.

### Setup

In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
# 아래 코드는 원하는 GPU 번호만 쓰도록 설정하는 코드
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

from functools import partial
import jax
import jax.numpy as jnp
import numpy as np
import optax
import transformers
from tqdm import trange


import lorax
import jax_gptq

gpu = jax.devices('gpu')[0]
cpu = jax.devices('cpu')[0]

## Toy Example

### Model/Data setup

First we'll define an MLP and make some parameters for it:

In [2]:
N_LAYER = 5
batch_size = 16
DIM = 256

def my_model(params, x):
  for layer in params:
    x = jax.nn.relu(x @ layer['w'] + layer['b'])

  return jnp.mean(x)

w_key, b_key, data_key = jax.random.split(jax.random.PRNGKey(0), 3)

w_keys = jax.random.split(w_key, N_LAYER)
b_keys = jax.random.split(b_key, N_LAYER)

# Make some params
params = [
    {
        'w': jax.random.normal(k1, (DIM, DIM)),
        'b': jax.random.normal(k2, (DIM,))
    }
    for k1, k2 in zip(w_keys, b_keys)
]


GPT-Q needs input data for quantization. For an actual model we'd use real data but here we'll just make some random inputs.

In [3]:
quant_data = [jax.random.normal(key, (batch_size, DIM)) for key in jax.random.split(data_key, 16)]

# We'll save an output for later comparison since the quantization process will delete the original params
original_output = my_model(params, quant_data[0])

### Run GPT-Q to get the quantized weights
That's all for the setup, we can now just run GPT-Q (without any changes to the original model code):

In [4]:
# Note that this may free the buffers associated with some or all of the parameters and the data to save VRAM
# I'd also recommend you put the params on the CPU, since `quantize()` will move the params to th GPU when necessary
quantized_params = jax_gptq.quantize(my_model, params, quant_data)

Quantizing: 0it [00:00, ?it/s]

Current env size: 2.62e+05 bytes
Current param env size: 0.00e+00 bytes


Quantizing: 2it [00:20, 10.19s/it]

Current env size: 2.62e+05 bytes
Current param env size: 1.02e+03 bytes


Quantizing: 3it [00:21,  6.54s/it]

...

Quantizing: 4it [00:23,  4.64s/it]

Current env size: 2.62e+05 bytes
Current param env size: 1.02e+03 bytes


Quantizing: 5it [00:24,  3.51s/it]

Current env size: 2.62e+05 bytes
Current param env size: 1.02e+03 bytes


Quantizing: 6it [00:26,  4.34s/it]


The matrices have been quantized but the biases have been left alone:

In [5]:
print(f'W type: {type(quantized_params[0]["w"])}')
print(f'B type: {type(quantized_params[0]["b"])}')

W type: <class 'jax_gptq.gptq.QuantizedMatrix'>
B type: <class 'jaxlib.xla_extension.ArrayImpl'>


**Note**: The quantization procedure depends on the parameter being used in a matrix multiplication. Currently JAX-GPTQ supports general dot operations (including ones using tensors with any number of dimensions larger than 1), and convolutions with kernels of spatial size 1.

### Applying the quantized weights
We can now run the quantized model without any code changes. All that's necessary is using `jax_gptq.use_quantized` to transform the function so it knows how to handle `QuantizedMatrix` values.

In [6]:
quantized_params = jax.device_put(quantized_params, gpu) # Move the params to the GPU

# Originally:
# my_model(params, inputs)
# After:
# jax_gptq(my_model)(params, inputs)
quant_output = jax_gptq.use_quantized(my_model)(quantized_params, quant_data[0])

print(f'Output of quantized network: {quant_output:.3e}')
print(f'Original output: {original_output:.3e}')

Output of quantized network: 8.691e+04
Original output: 8.575e+04


### Train with LoRA

Now that we've compressed our model to 4-bits (and change) per parameter, we can add full precision LoRA parameters for finetuning.

The one gotcha about combining the two is that Lorax doesn't know that QuantizedMatrix values are pytree leaves, so you need to give the Lorax functions an `is_leaf` predicate.

**Initialization:** The `init_lora` function expects a pytree describing which parameters should get LoRA parameters, which should be fully trained, and which should be left frozen. `lorax.simple_spec` is a helper function for making these specs.

In [7]:

def is_leaf(x):
    """
    #목적: 이 함수는 주어진 객체가 QuantizedMatrix의 인스턴스인지 확인합니다.
    # 이는 JAX의 tree_map 함수가 이 객체를 리프 노드로 인식하도록 하기 위함입니다.
    """
    return isinstance(x, jax_gptq.QuantizedMatrix)



"""
    목적: LoRA 사양을 정의합니다. 이는 양자화된 파라미터(quantized_params)에 대해 어떻게 LoRA 파라미터를 생성할지 결정합니다.
	•	decision_fn: 모든 파라미터에 대해 내적 랭크를 4로 지정합니다.
	•	tune_vectors: 편향 벡터를 조정 가능하도록 하지 않고, 고정된 파라미터 트리에 배치합니다.
	•	is_leaf: QuantizedMatrix 객체를 리프 노드로 인식합니다.
"""

lora_spec = lorax.simple_spec(
    
    params=quantized_params,
    decision_fn=lambda pytree_path, arr: 4, # Just ignore the inputs and specify an inner rank of 4 for all params# 모든 파라미터에 대해 내적 랭크를 4로 지정
    tune_vectors=False, # Tell Lorax to put all the biases in the frozen params tree instead of the tunable params tree
     # 모든 편향을 조정 가능한 파라미터 트리 대신 고정된 파라미터 트리에 배치
    is_leaf=is_leaf
)

# Lorax splits the parameters into two pytrees:
# freeze_params: Anything which received the value lorax.LORA_FREEZE in the spec
# train_params: Pairs of two narrow matrices for values which got positive integers as spec values, or the full parameter if the value lorax.LORA_FULL was in the spec
"""
•	목적: LoRA 파라미터를 초기화합니다. 이 함수는 파라미터를 두 개의 pytree로 나눕니다:
•	freeze_params: 고정된 파라미터. LoRA 사양에서 lorax.LORA_FREEZE 값을 받은 파라미터들.
•	train_params: 조정 가능한 파라미터. 사양에서 양의 정수 값을 받은 파라미터들.
"""

freeze_params, train_params = lorax.init_lora(param_tree = quantized_params, spec = lora_spec, rng =  jax.random.PRNGKey(1234), is_leaf=is_leaf)

def merge_quantized_with_lora(q_params, lora_freeze):
    return jax.tree_map(
        lambda quant, from_lora: quant if isinstance(quant, jax_gptq.QuantizedMatrix) else from_lora,
        q_params,
        lora_freeze,
        is_leaf=lambda x: isinstance(x, jax_gptq.QuantizedMatrix) # Tell tree_map to treat QuantizedMatrix as a single value instead of a non-leaf node
    )
# Now we put the actual quantized params back
#freeze_params = merge_quantized_with_lora(quantized_params, freeze_params)

The `lorax.lora` transform converts a function from expecting a single pytree in the specified argument to expecting a tuple of two pytrees. It composes with other JAX transforms such as `jax_gptq.use_quantized`, so we can use both at once with no modifications to our model code.

In [8]:
combined_params = (freeze_params, train_params)

my_model_with_lora_and_quantized_weights = jax_gptq.use_quantized(lorax.lora(my_model))

# The differences from the original `my_model` function are:
# 1. The params argument now expects a tuple of (frozen_params, trainable_params)
# 2. It knows how to compute with quantized weights
quantized_plus_lorax_output = my_model_with_lora_and_quantized_weights(combined_params, quant_data[0])

print(f'GPTQ + Lorax output: {quantized_plus_lorax_output:.3e}')
print(f'GPTQ only: {quant_output:.3e}')

GPTQ + Lorax output: 8.691e+04
GPTQ only: 8.691e+04


The above values are identical since LoRA initializes one of each pair of matrices as zeros.

Let's look at the size of each pytree:

In [9]:
count_params = partial(jax.tree_util.tree_reduce,
  lambda acc, param: acc + (param.size if isinstance(param, jnp.ndarray) else 0),
  initializer=0
)

print(f'{count_params(freeze_params):.3e} frozen params')
print(f'{count_params(train_params):.3e} trainable params')

1.677e+05 frozen params
1.024e+04 trainable params


Training with this function is no different from any other JAX function, just make sure to only differentiate your loss with respect to the trainable parameters only. (See the next section for an example).

## GPT-Q-ing + LoRA-ing HuggingFace's Flax GPT-2
I developed these transforms for use with my Haiku models, but since all JAX models are pure functions at the end of the day, it shouldn't matter what framework you use. Lorax supports matmuls and other matmul-like operations such as embedding lookups and 1-D convs.

This is a minimal example of applying the combination to `gpt2-medium`, but it's basically model agnostic.

First let's get the model:

In [10]:
from transformers import AutoTokenizer, FlaxAutoModelForCausalLM

In [11]:
model_name = 'gpt2-medium'
# 사전 학습된 모델에서 토크나이저를 불러옵니다. 이 토크나이저는 텍스트를 토큰으로 변환하는 데 사용됩니다.
tokenizer = AutoTokenizer.from_pretrained(model_name)
#사전 학습된 모델을 Flax 형식으로 불러옵니다. _do_init=False는 모델을 초기화하지 않고 불러옵니다. 이는 이후 양자화를 위해 파라미터를 별도로 처리하기 위함입니다.
model, params = FlaxAutoModelForCausalLM.from_pretrained(model_name, _do_init=False)
#모델 파라미터를 CPU 장치에 배치합니다. 이는 GPU 메모리 사용을 줄이기 위해 초기 단계에서 파라미터를 CPU에 배치하는 것입니다.
params = jax.device_put(params, cpu)

#	•	params['transformer']['wte']['embedding']: GPT-2 모델의 임베딩 테이블입니다. 입력 토큰을 임베딩 벡터로 변환하는 데 사용됩니다.
#	•	np.asarray(...): 임베딩 테이블을 numpy 배열로 변환하여 저장합니다. 이는 양자화 과정에서 임베딩 테이블이 손상되는 것을 방지하기 위함입니다. 양자화 후에 임베딩 테이블을 다시 사용할 수 있도록 별도로 저장해 둡니다.
# Because the embedding table is reused as the output linear layer, it'll get quantized at the end of the process, but that will seriously screw up the embedding lookup step, so we'll just save it for later here
orig_embedding_table = np.asarray(params['transformer']['wte']['embedding'])

In [4]:
print(type(params))

<class 'dict'>


The GPT-Q paper used real text data for quantization, but for this demo I'll just generate some random values.

In [12]:
QUANT_BATCH_SIZE = 4 #	•	QUANT_BATCH_SIZE: 양자화를 위해 사용할 배치 크기입니다. 여기서는 4로 설정되어 있습니다.
#양자화 예제의 길이입니다. 각 예제는 64개의 토큰으로 구성됩니다. 이 값을 더 크게 설정할 수 있지만, Colab에서 메모리 충돌을 방지하기 위해 작은 값으로 설정되었습니다
QUANT_EXAMPLE_LENGTH = 64 # I'd recommend making this bigger, but needs to be small to not crash colab

quantization_data = []
key = jax.random.PRNGKey(0) #JAX의 랜덤 키를 초기화합니다. 랜덤 키는 재현 가능한 무작위 값을 생성하는 데 사용됩니다.
for _ in range(32):
  #jax.random.randint(key, (QUANT_BATCH_SIZE, QUANT_EXAMPLE_LENGTH), 0, 50256): 무작위 정수로 구성된 텐서를 생성합니다. 각 배치는 QUANT_BATCH_SIZE x QUANT_EXAMPLE_LENGTH 크기의 텐서입니다. 각 값은 0에서 50255 사이의 정수입니다 (50256은 GPT-2의 단어 집합 크기입니다).
  batch = jax.random.randint(key, (QUANT_BATCH_SIZE, QUANT_EXAMPLE_LENGTH), 0, 50256)
  quantization_data.append(batch) #quantization_data.append(batch): 생성된 배치를 양자화 데이터 리스트에 추가합니다.
  key, = jax.random.split(key, 1) #랜덤 키를 업데이트하여 다음 배치를 생성할 때 사용할 새로운 키를 생성합니다.


In [7]:
print(type(quantization_data[0]))

<class 'jaxlib.xla_extension.ArrayImpl'>


HuggingFace's models don't have quite the right call signature, so we'll make a wrapper which takes (params, inputs) as an argument:
주어진 코드는 HuggingFace 모델의 호출 시그니처를 변경하여 JAX 기반의 양자화 함수에 맞게 래퍼를 작성하고, 양자화된 모델 파라미터를 원래의 임베딩 테이블로 대체하는 과정을 포함합니다

In [13]:
# HuggingFace의 모델 호출 시그니처가 JAX의 요구 사항과 맞지 않기 때문에,
# (params, inputs) 형식의 인수를 받도록 래퍼를 작성합니다.
def apply_model(params, batch):
  return model(batch, params=params)
# 작성한 래퍼 함수와 함께 GPT-Q 양자화 함수를 호출하여 양자화된 파라미터를 생성합니다.
#	•	jax_gptq.quantize 함수 호출: 작성한 래퍼 함수(apply_model), 모델 파라미터(params), 양자화 데이터(quantization_data)를 사용하여 GPT-Q 알고리즘을 통해 모델 파라미터를 양자화합니다.
# •	quantized_params: 양자화된 파라미터가 저장됩니다.
quantized_params = jax_gptq.quantize(apply_model, params, quantization_data)

Quantizing: 0it [00:00, ?it/s]/home/AI2/anaconda3/envs/12jax/lib/python3.12/site-packages/jax_gptq/quantize_interpreter.py:352: UserWarning: Operation gather not supported for quantization
  warnings.warn(f'Operation {eqn.primitive.name} not supported for quantization')


Current env size: 3.28e+04 bytes
Current param env size: 2.23e+08 bytes


Quantizing: 2it [00:38, 19.43s/it]

Current env size: 1.68e+08 bytes
Current param env size: 4.21e+06 bytes


Quantizing: 3it [01:13, 25.75s/it]

...

Current env size: 1.69e+08 bytes
Current param env size: 1.68e+07 bytes


Quantizing: 97it [20:13, 12.27s/it]

Current env size: 6.79e+07 bytes
Current param env size: 2.06e+08 bytes


E0724 18:44:17.280217 2540469 pjrt_stream_executor_client.cc:2985] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Failed to allocate request for 49.08MiB (51463168B) on device ordinal 0


ValueError: RESOURCE_EXHAUSTED: Failed to allocate request for 49.08MiB (51463168B) on device ordinal 0

In [14]:
# Replace the quantized embedding table with the original one
#양자화된 임베딩 테이블을 원래의 임베딩 테이블로 대체합니다.
#	•	목적: 임베딩 테이블은 입력 토큰을 벡터로 변환하는 중요한 역할을 합니다. 양자화 과정에서 임베딩 테이블이 손상될 수 있으므로, 원래의 임베딩 테이블로 대체하여 정확성을 유지합니다.
#   •	quantized_params['transformer']['wte']['embedding']: 양자화된 파라미터에서 임베딩 테이블을 찾아서 원래의 임베딩 테이블(orig_embedding_table)로 대체합니다.
quantized_params['transformer']['wte']['embedding'] = jnp.asarray(orig_embedding_table)
quantized_params = jax.device_put(quantized_params, gpu)

### Finetuning GPT-2 with Lorax

Same as [above](https://colab.research.google.com/drive/18rkULbWqk7mNZDx7Scx-JS3p_s45mgok#scrollTo=HKkhcjx9zJy6&line=3&uniqifier=1), we get the original param structure to tell Lorax how to initialize the LoRA params, then merge the quantized params back in after.

이 코드는 LoRA(Low-Rank Adaptation) 기법을 사용하여 양자화된 GPT-2 모델을 미세 조정하는 과정입니다. LoRA는 모델의 일부 파라미터만을 저순위 행렬로 미세 조정함으로써, 메모리 사용량과 계산량을 줄이는 기법입니다. 

In [15]:
# Get pre-quantization param tree (some nodes will just be abstract values)
orig_params_or_shapes = jax_gptq.utils.quantized_params_to_shaped_arrays(quantized_params)

# Tell Lorax which leaves should be frozen/fully trained/LoRA trained
# LoRA 사양을 정의하여 어느 파라미터가 고정되고, 완전 훈련되며, LoRA로 훈련될지 지정합니다.
#	•	simple_spec 함수:
#   •	orig_params_or_shapes: 원래 파라미터 구조.
#   •	lambda path, arr: 경로에 c_attn 또는 mlp 패턴이 포함된 경우 내적 랭크를 16으로 설정하고, 그렇지 않으면 파라미터를 고정합니다.
#   •	tune_vectors=True: 벡터를 조정하도록 설정합니다.

spec = lorax.simple_spec(
    orig_params_or_shapes,
    lambda path, arr: 16 if any(pattern in path for pattern in ['c_attn', 'mlp']) else lorax.LORA_FREEZE,
    tune_vectors=True
)

# Initialize parameters
"""
	•	목적: LoRA 파라미터를 초기화합니다.
	•	jax.random.split 함수: 랜덤 키를 분할하여 초기화 키를 생성합니다.
	•	init_lora 함수:
	•	orig_params_or_shapes: 원래 파라미터 구조.
	•	spec: 정의된 LoRA 사양.
	•	init_key: 파라미터 초기화에 사용할 랜덤 키.

"""
key, init_key = jax.random.split(key)
freeze_params, train_params = lorax.init_lora(
    orig_params_or_shapes,
    spec,
    init_key
)
# 양자화된 파라미터를 고정된 파라미터 트리에 다시 병합합니다.
# Put the quantized params back into the frozen param tree
freeze_params = merge_quantized_with_lora(quantized_params, freeze_params)
combined_params = freeze_params, train_params

/tmp/ipykernel_1318360/3777058010.py:38: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(


Now we can just transform the `apply_model` function and it will use both LoRA and 4-bit quantized parameters

In [16]:
"""
	•	목적: apply_model 함수를 LoRA와 4-bit 양자화된 파라미터를 모두 사용하도록 변환합니다.
	•	lorax.lora(apply_model): LoRA 파라미터를 사용하는 함수로 변환합니다.
	•	jax_gptq.use_quantized: 4-bit 양자화된 파라미터를 사용하는 함수로 변환합니다.
"""

quantized_plus_lora_fn = jax_gptq.use_quantized(lorax.lora(apply_model))

### Training
Training isn't actually any different from normal training, since you can just think of `freeze_params` as being a constant argument, but here's a demo for completness.

First I'll define a toy corpus which demonstrates Alan's love of cats and Grace's dislike of them.

In [17]:
# 고양이와 개를 좋아하는 인물에 대한 간단한 텍스트 데이터셋을 만듭니다.
CATS = ['lions', 'tigers', 'cheetahs', 'cats', 'ocelots', 'kittens']
DOGS = ['wolves', 'dogs', 'coyotes', 'huskies', 'poodles', 'puppies']

CAT_LOVER = 'Alan'
DOG_LOVER = 'Grace'

dataset = []
for name, polarity in [(CAT_LOVER, True), (DOG_LOVER, False)]:
  liked, disliked = (CATS, DOGS) if polarity else (DOGS, CATS)
  for kind in liked:
    dataset.append(f'{name}: {kind}? I love them!')
    dataset.append(f'{name}: Hey look at those {kind}, that\'s pretty cool')

  for kind in disliked:
    dataset.append(f'{name}: {kind}? I hate them!')
    dataset.append(f'{name}: Oh no, some {kind}! How scary!')
# 텍스트 데이터를 토큰화하고, 최대 길이에 맞춰 패딩합니다.
tokenized_data = [jnp.asarray(tokenizer.encode(ex)) for ex in dataset]
max_len = max(ex.shape[0] for ex in tokenized_data)
# Pad the data to speed up jitting. Not worrying about masking due to laziness.
# 패딩을 통해 데이터를 동일한 길이로 맞춥니다. 마스킹은 생략합니다.
tokenized_data = [jnp.pad(ex, (0, max_len - ex.shape[0])) for ex in tokenized_data]
"""
	•	목적: 변환된 모델을 JIT 컴파일하여 성능을 최적화합니다.
	•	jax.jit: JAX 함수의 JIT 컴파일러를 사용하여 모델을 컴파일합니다.
"""
jitted_model = jax.jit(quantized_plus_lora_fn)


In [18]:
"""
	•	목적: 주어진 프리픽스를 사용하여 모델의 예측을 출력하는 함수를 정의합니다.
	•	tokenizer.encode: 프리픽스를 토큰화합니다.
	•	jitted_model(params, tokens[None]): JIT 컴파일된 모델을 사용하여 로짓(logits)을 계산합니다.
	•	jax.nn.log_softmax: 로짓에 소프트맥스 함수를 적용하여 확률 분포를 계산합니다.
	•	jax.lax.top_k: 상위 5개의 예측 단어와 그 확률을 찾습니다.
	•	예측 결과 출력: 프리픽스에 대한 예측 결과를 출력합니다.

"""
def make_prediction(params, prefix):
  tokens = jnp.asarray(tokenizer.encode(prefix))
  logits = jitted_model(params, tokens[None]).logits
  
  logprobs = jnp.exp(jax.nn.log_softmax(logits[0, -1]))
  pred_probs, pred_words = jax.lax.top_k(logprobs, 5)

  print(f'Predictions for: "{prefix}"')
  for i, (word_id, prob) in enumerate(zip(pred_words, pred_probs), 1):
    print(f'{i}. {tokenizer.decode([word_id])} - {prob:.2%}')
  print()

test_examples = [
    f'{CAT_LOVER}: jaguars? I',
    f'{DOG_LOVER}: jaguars? I'
]

Let's look at the next word predictions of the unmodified model:

In [19]:
for ex in test_examples:
  make_prediction(combined_params, ex)

2024-07-23 11:32:30.118895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-23 11:32:30.192666: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-23 11:32:30.193099: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-23 11:32:31.438438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Predictions for: "Alan: jaguars? I"
1. 'm - 11.25%
2.  mean - 8.90%
3. 've - 7.17%
4.  don - 6.74%
5.  thought - 4.55%

Predictions for: "Grace: jaguars? I"
1. 'm - 10.07%
2.  don - 7.90%
3.  mean - 6.11%
4. 've - 6.09%
5.  thought - 4.36%



Next we set up a standard training loop. The only difference is that we keep the train/freeze params separate for the optimizer. There's no differences needed for the quantization.

I'll just train with a batch size of 1 here since I don't want to bother with masking, but the transformed model function is fully compatible with vmap etc.

In [20]:
def loss_fn(train_params, freeze_params, seq):
  inputs = seq[:-1]
  targets = seq[1:]

  combined_params = (freeze_params, train_params)
  logits = quantized_plus_lora_fn(combined_params, inputs[None]).logits[0]
  logprobs = jax.nn.log_softmax(logits)
  losses = -jnp.take_along_axis(logprobs, targets[:, None], axis=-1)
  return jnp.mean(losses)

optimizer = optax.adamw(learning_rate=1e-4, weight_decay=1e-4)
opt_state = optimizer.init(combined_params[1])

@jax.jit
def update_fn(combined_params, opt_state, example):
  freeze_params, train_params = combined_params

  # The main thing is that we have to split up the params here so that JAX knows what to differentiate with respect to
  loss, grads = jax.value_and_grad(loss_fn)(train_params, freeze_params, example)

  updates, opt_state = optimizer.update(grads, opt_state, params=train_params)
  new_train_params = optax.apply_updates(train_params, updates)
  return (freeze_params, new_train_params), opt_state, loss

In [21]:
bar = trange(50)
for epoch in bar:
  key, = jax.random.split(key, 1)
  permutation = jax.random.permutation(key, jnp.arange(len(dataset)))
  total_loss = 0
  for index in permutation:
    example = tokenized_data[index]
    combined_params, opt_state, loss = update_fn(combined_params, opt_state, example)
    total_loss += loss
  bar.set_description(f'Epoch {epoch} - Loss: {total_loss / len(tokenized_data):.3e}')

Epoch 49 - Loss: 2.581e-01: 100%|██████████| 50/50 [34:29<00:00, 41.38s/it] 


The trained LoRA parameters give us a model which predicts that Alan will love jaguars, and Grace will hate them:

In [22]:
for example in test_examples:
  make_prediction(combined_params, example)
  print()

Predictions for: "Alan: jaguars? I"
1.  love - 83.57%
2.  hate - 16.42%
3.  LOVE - 0.00%
4.  like - 0.00%
5.  want - 0.00%


Predictions for: "Grace: jaguars? I"
1.  hate - 62.58%
2.  love - 37.39%
3.  LOVE - 0.01%
4. 'll - 0.01%
5.  Hate - 0.01%


